# Quick Jewelry Generation Demo
## Reproducing Optimal Configuration in <10 Minutes

This notebook demonstrates the key improvements from the research:
- **Compel prompt weighting** for better jewelry term adherence
- **Optimal parameter selection** via human evaluation
- **Consistent modern aesthetics** matching target brands

**Expected runtime**: 5-8 minutes on CPU, 2-3 minutes on GPU


In [ ]:
# Install required packages (run once)
# %pip install torch diffusers transformers compel matplotlib pillow


In [ ]:
import torch
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
from compel import Compel
import time
from datetime import datetime
import os

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


## 🏆 Optimal Configuration (from Human Evaluation)

Based on systematic testing and human evaluation:
- **Model**: SD 1.5 (best balance of quality and speed)
- **Sampler**: Euler Ancestral (best quality/speed balance)
- **Strategy**: medium_compel (1.2x weight on jewelry terms)
- **CFG Scale**: 9.0 (optimal prompt adherence)
- **Steps**: 20 (sufficient quality, good speed)

**Key Finding**: AI metrics (CLIP, LAION) poorly predicted human preferences (38.4%, 51.4% accuracy)


In [ ]:
def setup_pipeline(device=device):
    """Setup the optimized SD 1.5 pipeline"""
    print("🔧 Loading SD 1.5 pipeline...")
    
    dtype = torch.float16 if device == "cuda" else torch.float32
    
    # Load SD 1.5 with optimal configuration
    pipeline = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=dtype,
        variant="fp16" if device == "cuda" else None
    )
    
    # Set optimal sampler (from human evaluation)
    pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(
        pipeline.scheduler.config
    )
    
    pipeline.to(device)
    
    # Memory optimizations
    if device == "cpu":
        pipeline.enable_attention_slicing()
    else:
        pipeline.enable_memory_efficient_attention()
    
    # Setup Compel for prompt weighting
    compel = Compel(
        tokenizer=pipeline.tokenizer,
        text_encoder=pipeline.text_encoder
    )
    
    print(f"✅ Pipeline ready on {device}")
    return pipeline, compel

# Load the pipeline
pipeline, compel = setup_pipeline()
